In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
from gpfa_imputation.imputation import *
from gpfa_imputation.data_preparation import *

import torch
import pandas as pd
import numpy as np
from pathlib import Path
from pyprojroot import here
import matplotlib.pyplot as plt

from gpfa_imputation.utils import cache_disk

In [ ]:
def reset_seed():
    torch.manual_seed(27);
    np.random.seed(27);

In [ ]:
torch.manual_seed(27);
np.random.seed(27);
cache = True
cache_path = here() / ".cache"

## Load Data

take the first 200 rows from the Hainich dataset

In [ ]:
hai_path = Path("FLX_DE-Hai_FLUXNET2015_FULLSET_HH_2000-2012_1-4.csv")
hai_raw = pd.read_csv(here("data") / hai_path, na_values=["-9999", "-9999.99"], parse_dates=[0, 1], nrows=200)

In [ ]:
meteo_vars = {
    "TA_F": "TA",
    "SW_IN_F": "SW_IN",
    "LW_IN_F": "LW_IN",
    "VPD_F": "VPD",
    #"PA": "PA"
}

units = {
    'TA': '°C',
    'SW_IN': 'W m-2',
    'LW_IN': 'W m-2',
    'VPD': 'hPa'
}

hai = (hai_raw
       .rename(columns=meteo_vars)
       .set_index("TIMESTAMP_END")
       .loc[:, meteo_vars.values()])
hai

,TA,SW_IN,LW_IN,VPD
TIMESTAMP_END,,,,
2000-01-01 00:30:00,-0.60,0.0,302.475,0.222
2000-01-01 01:00:00,-0.65,0.0,302.475,0.122
2000-01-01 01:30:00,-0.58,0.0,301.677,0.090
2000-01-01 02:00:00,-0.51,0.0,301.677,0.110
2000-01-01 02:30:00,-0.49,0.0,301.677,0.102
...,...,...,...,...
2000-01-05 02:00:00,4.74,0.0,330.202,1.191
2000-01-05 02:30:00,4.75,0.0,330.202,1.057
2000-01-05 03:00:00,4.76,0.0,330.202,0.935


In [ ]:
reset_seed()
data = GPFADataTest(hai[:150]).add_random_missing()

In [ ]:
imp = GPFAImputationExplorer(data.data, latent_dims=2)

In [ ]:
imp

GPFA Imputation:
    N obs: 150
    N features 4 (TA, SW_IN, LW_IN, VPD)
    N missing observations 168
    N latent: 2

In [ ]:
# imp.learner.model.covar_module.Lambda = torch.nn.Parameter(torch.ones(4,2))

# imp.fit_predict()

# res = imp.to_result(data.data_compl_tidy, units=units)
# res.display_results()

###  Compute

In [ ]:
cache = here() / ".cache" / "hai_test_init_values.pickle"

In [ ]:
@cache_disk(cache)
def compute():
    out = {}
    
    imp = GPFAImputationExplorer(data.data, latent_dims=2)
    out["normal"] = imp.fit().to_result(data.data_compl_tidy, units=units)
    
    imp = GPFAImputationExplorer(data.data, latent_dims=2)
    imp.learner.model.covar_module.Lambda = torch.nn.Parameter(torch.ones(4,2))
    out["ones"] = imp.fit().to_result(data.data_compl_tidy, units=units)
    
    imp = GPFAImputationExplorer(data.data, latent_dims=2)
    imp.learner.model.covar_module.Lambda = torch.nn.Parameter(torch.zeros(4,2))
    out["zeros"] = imp.fit().to_result(data.data_compl_tidy, units=units)
    
    imp = GPFAImputationExplorer(data.data, latent_dims=2)
    imp.learner.model.covar_module.Lambda = torch.nn.Parameter(torch.rand(4,2))
    out["rand1"] = imp.fit().to_result(data.data_compl_tidy, units=units)
    
    imp = GPFAImputationExplorer(data.data, latent_dims=2)
    imp.learner.model.covar_module.Lambda = torch.nn.Parameter(torch.rand(4,2))
    out["rand2"] = imp.fit().to_result(data.data_compl_tidy, units=units)
    
    imp = GPFAImputationExplorer(data.data, latent_dims=2)
    imp.learner.model.covar_module.Lambda = torch.nn.Parameter(torch.rand(4,2))
    out["rand3"] = imp.fit().to_result(data.data_compl_tidy, units=units)
    
    return out

In [ ]:
results = compute()

In [ ]:
results["normal"].plot_pred()

/home/simone/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [ ]:
results["normal"].display_results()

/home/simone/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [ ]:
results["ones"].display_results()

/home/simone/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [ ]:
results["zeros"].display_results()

/home/simone/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [ ]:
results["rand1"].display_results()

/home/simone/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [ ]:
results["rand2"].display_results()

/home/simone/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [ ]:
results["rand3"].display_results()

/home/simone/.local/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)